# 01. 선형 회귀 (Linear Regression)

## 학습 목표
- 선형 회귀의 원리 이해
- scikit-learn으로 모델 구현
- 모델 평가 지표 (MSE, R²) 이해

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import make_regression

# 한글 폰트 설정 (선택)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

## 1. 데이터 생성

In [ ]:
# 인공 데이터 생성
np.random.seed(42)
X, y = make_regression(n_samples=100, n_features=1, noise=15, random_state=42)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
# 데이터 시각화
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.7, edgecolors='black')
plt.xlabel('Feature (X)')
plt.ylabel('Target (y)')
plt.title('Linear Regression Data')
plt.grid(True, alpha=0.3)
plt.show()

## 2. 데이터 분할

In [ ]:
# Why: 80:20 split is a common default — enough test data for reliable evaluation
# while retaining most data for training. random_state ensures reproducibility.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## 3. 모델 학습

In [ ]:
# 선형 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 모델 파라미터 확인
print(f"기울기 (Coefficient): {model.coef_[0]:.4f}")
print(f"절편 (Intercept): {model.intercept_:.4f}")
print(f"\n회귀식: y = {model.coef_[0]:.4f} * x + {model.intercept_:.4f}")

## 4. 예측 및 평가

In [ ]:
# 예측
y_pred = model.predict(X_test)

# Why: MSE penalizes large errors quadratically, making it sensitive to outliers.
# RMSE converts back to original units for interpretability.
# R² tells what fraction of variance is explained — the closer to 1, the better.
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("=== 모델 평가 ===")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"RMSE (Root MSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 회귀선
axes[0].scatter(X_test, y_test, alpha=0.7, label='Actual', edgecolors='black')
axes[0].plot(X_test, y_pred, color='red', linewidth=2, label='Predicted')
axes[0].set_xlabel('Feature (X)')
axes[0].set_ylabel('Target (y)')
axes[0].set_title('Linear Regression - Test Data')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Why: Residuals should be randomly scattered around zero. Any pattern (funnel,
# curve) indicates violated assumptions like heteroscedasticity or non-linearity.
residuals = y_test - y_pred
axes[1].scatter(y_pred, residuals, alpha=0.7, edgecolors='black')
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_xlabel('Predicted Values')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 다중 선형 회귀

In [ ]:
# 다중 특성 데이터 생성
X_multi, y_multi = make_regression(
    n_samples=200, 
    n_features=3, 
    noise=10, 
    random_state=42
)

# DataFrame으로 변환
df = pd.DataFrame(X_multi, columns=['Feature_1', 'Feature_2', 'Feature_3'])
df['Target'] = y_multi
print(df.head())
print(f"\nShape: {df.shape}")

In [ ]:
# 다중 회귀 모델 학습
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42
)

model_multi = LinearRegression()
model_multi.fit(X_train_m, y_train_m)

# 결과
print("=== 다중 선형 회귀 ===")
print(f"Coefficients: {model_multi.coef_}")
print(f"Intercept: {model_multi.intercept_:.4f}")

y_pred_m = model_multi.predict(X_test_m)
print(f"\nR² Score: {r2_score(y_test_m, y_pred_m):.4f}")

## 6. 실제 데이터 예제 (Boston Housing 대체)

sklearn의 Boston Housing 데이터셋은 deprecated되었으므로 California Housing을 사용합니다.

In [ ]:
from sklearn.datasets import fetch_california_housing

# 데이터 로드
housing = fetch_california_housing()
X_housing = housing.data
y_housing = housing.target

print(f"Features: {housing.feature_names}")
print(f"Shape: {X_housing.shape}")
print(f"Target: Median house value (in $100,000s)")

In [ ]:
# 데이터 분할 및 학습
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=42
)

model_housing = LinearRegression()
model_housing.fit(X_train_h, y_train_h)

y_pred_h = model_housing.predict(X_test_h)

print("=== California Housing 회귀 결과 ===")
print(f"R² Score: {r2_score(y_test_h, y_pred_h):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_h, y_pred_h)):.4f}")

In [ ]:
# Why: Linear regression coefficients show each feature's marginal effect on the
# target, but only meaningful when features are on comparable scales. Large
# coefficients on unscaled data may just reflect unit differences, not importance.
importance = pd.DataFrame({
    'Feature': housing.feature_names,
    'Coefficient': model_housing.coef_
}).sort_values('Coefficient', key=abs, ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(importance['Feature'], importance['Coefficient'])
plt.xlabel('Coefficient')
plt.title('Feature Coefficients - California Housing')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 정리

### 핵심 개념
- **선형 회귀**: y = wx + b 형태의 선형 관계 학습
- **경사 하강법**: 손실 함수(MSE)를 최소화하는 방향으로 파라미터 업데이트
- **R² Score**: 모델이 데이터의 분산을 얼마나 설명하는지 (0~1, 높을수록 좋음)

### 다음 단계
- 다항 회귀 (Polynomial Regression)
- 정규화 (Ridge, Lasso)
- 특성 스케일링의 중요성